In [1]:
import os
import time
import datetime
import pandas as pd
import math
import asyncio
import requests

#環境情報色々
DRIVER_PATH = '/Users/hayashiryuutarou/Desktop/study/webcrow/selenium/chromedriver'

#各種Keyの読み込み
import configparser
#config取得
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')
ACCOUNT = config_ini["SBI"]["ACCOUNT"]
PASSWORD = config_ini["SBI"]["PASSWORD"]
name = config_ini["SBI"]["name"]
args = "マザーズ"

In [2]:
#対象のデータ選定
import pandas as pd
stock_list = pd.read_csv("../../stock/stocklist.csv")
#対象の銘柄一覧を取得する
targets = stock_list.loc[stock_list["市場名"]==args]
length = math.ceil(len(targets["市場名"])/80)
target_lists = []
for i in range(length):
    start = i*80
    end = (i+1)*80
    target_lists.append(targets[start:end])

In [3]:
import SBI_getdata as getdata

In [4]:
'''
drivers = []

for i in range(len(target_lists)):
    
    params ={
    "ACCOUNT":ACCOUNT,
    "PASSWORD":PASSWORD,
    "name":name,
    "DRIVER_PATH":DRIVER_PATH,
    "TARGET":target_lists[i]
    }
    
    drivers.append(getdata.set_stock_info(params))
'''

'\ndrivers = []\n\nfor i in range(len(target_lists)):\n    \n    params ={\n    "ACCOUNT":ACCOUNT,\n    "PASSWORD":PASSWORD,\n    "name":name,\n    "DRIVER_PATH":DRIVER_PATH,\n    "TARGET":target_lists[i]\n    }\n    \n    drivers.append(getdata.set_stock_info(params))\n'

In [5]:
#https://note.com/hagure_melon/n/ndf2f05bc791c
#できればこっちの形に書き換える

async def handler(loop):
    async def async_set_stock_info(i):
        async with asyncio.Semaphore(20):
            return await loop.run_in_executor(None, getdata.set_stock_info, params)
    tasks = [getdata.async_set_stock_info(params) for i in len(target_lists)]
    return await asyncio.gather(*tasks)
loop = asyncio.get_event_loop()
data = loop.run_until_complete(handler(loop))
loop.close()

RuntimeError: This event loop is already running